# FairBatch on the Synthetic Data

#### This Jupyter Notebook simulates FairBatch on the synthetic data.
#### It includes three fairness metrics: equal opportunity, equalized odds, and demographic parity.

## Import libraries

In [1]:
import sys, os
import numpy as np
import math
import random
import itertools
import copy

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch

from models import LogisticRegression, weights_init_normal, test_model
from FairBatchSampler import FairBatch, CustomDataset

import warnings
warnings.filterwarnings("ignore")

## Load and process the data

In [3]:
xz_train = np.load('./synthetic_data/xz_train.npy')
y_train = np.load('./synthetic_data/y_train.npy') 
z_train = np.load('./synthetic_data/z_train.npy')

xz_test = np.load('./synthetic_data/xz_test.npy')
y_test = np.load('./synthetic_data/y_test.npy') 
z_test = np.load('./synthetic_data/z_test.npy')

xz_train = torch.FloatTensor(xz_train)
y_train = torch.FloatTensor(y_train)
z_train = torch.FloatTensor(z_train)

xz_test = torch.FloatTensor(xz_test)
y_test = torch.FloatTensor(y_test)
z_test = torch.FloatTensor(z_test)

In [4]:
print("---------- Number of Data ----------" )
print(
    "Train data : %d, Test data : %d "
    % (len(y_train), len(y_test))
)       
print("------------------------------------")

---------- Number of Data ----------
Train data : 2000, Test data : 1000 
------------------------------------


## Training function

In [4]:
def run_epoch(model, train_features, labels, optimizer, criterion):
    """Trains the model with the given train data.

    Args:
        model: A torch model to train.
        train_features: A torch tensor indicating the train features.
        labels: A torch tensor indicating the true labels.
        optimizer: A torch optimizer.
        criterion: A torch criterion.

    Returns:
        loss value.
    """
    
    optimizer.zero_grad()

    label_predicted = model.forward(train_features)
    loss  = criterion((F.tanh(label_predicted.squeeze())+1)/2, (labels.squeeze()+1)/2)
    loss.backward()

    optimizer.step()
    
    return loss.item()

In [45]:
from torchmetrics.functional.pairwise import pairwise_euclidean_distance

def select_k_similar_pairs(x1, x2, k):
    distances = pairwise_euclidean_distance(x1, x2)

    flatten_distances = distances.flatten()

    top_k_indices = torch.topk(flatten_distances, k, largest=False)[1]

    selected_x1 = torch.zeros((k, x1.size()[1]))
    selected_x2 = torch.zeros((k, x2.size()[1]))

    for i in range(k):
        selected_x1[i] = x1[int(top_k_indices[i] // distances.size()[1])]
        selected_x2[i] = x2[int(top_k_indices[i] % distances.size()[1])]

    return selected_x1, selected_x2

# 1. FairBatch w.r.t. Equal Opportunity
### The results are in Section 4.1 of the paper.

In [77]:
full_tests = []

# Set the train data
train_data = CustomDataset(xz_train, y_train, z_train)

seeds = [0,1,2,3,4,5,6,7,8,9]
for seed in seeds:
    
    print("< Seed: {} >".format(seed))
    
    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------
    
    #model = LogisticRegression(3,1)
    model = nn.Sequential(
        nn.Linear(3, 32),
        nn.ReLU(),
        nn.Linear(32, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
    )

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    # ---------------------
    #  Define FairBatch and DataLoader
    # ---------------------

    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = 0.005, target_fairness = 'eqopp', replacement = False, seed = seed)
    #sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = 0.005, target_fairness = 'original', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    # ---------------------
    #  Model training
    # ---------------------
    for epoch in range(300):

        tmp_loss = []
        
        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)
            
        losses.append(sum(tmp_loss)/len(tmp_loss))
        
    tmp_test = test_model(model, xz_test, y_test, z_test)
    
    x1 = xz_test[(z_test == 0.0) & (y_test == 1.0)]
    x2 = xz_test[(z_test == 1.0) & (y_test == 1.0)]
    
    #min_num = min(x1.size()[0], x2.size()[0])
    #x1 = x1[:min_num]
    #x2 = x2[:min_num]
    
    x1_t = model[0](x1)
    x2_t = model[0](x2)
    
    #diff = torch.sqrt(torch.sum((x1_t - x2_t) ** 2))
    diff = torch.max(pairwise_euclidean_distance(x1_t, x2_t))
    
    tmp_test["diff"] = diff
    
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, EO disparity: {}".format(tmp_test['Acc'], tmp_test['EO_Y1_diff']))
    print("----------------------------------------------------------------------")

< Seed: 0 >
  Test accuracy: 0.8700000047683716, EO disparity: 0.040665756528306285
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8690000176429749, EO disparity: 0.040665756528306285
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8679999709129333, EO disparity: 0.036812191981100195
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8700000047683716, EO disparity: 0.04142097772097486
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8619999885559082, EO disparity: 0.031031845160290894
----------------------------------------------------------------------
< Seed: 5 >
  Test accuracy: 0.8679999709129333, EO disparity: 0.038738974254703185
----------------------------------------------------------------------
< Seed: 6 >
  Test accuracy: 0.8629999756813049, EO disparity: 0.017883

In [78]:
tmp_acc = []
tmp_eo = []
tmp_diff = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_eo.append(full_tests[i]['EO_Y1_diff'])
    tmp_diff.append(full_tests[i]["diff"])

print("Test accuracy (avg): {}".format(sum(tmp_acc)/len(tmp_acc)))
print("EO disparity  (avg): {}".format(sum(tmp_eo)/len(tmp_eo)))
print("Output F Norm = {}".format(sum(tmp_diff) / len(tmp_diff)))

Test accuracy (avg): 0.8671999931335449
EO disparity  (avg): 0.03557285463928507
Output F Norm = 19.98434066772461


# 2. FairBatch w.r.t. Equalized Odds 
### The results are in the supplementary of the paper.

In [7]:
full_tests = []

# Set the train data
train_data = CustomDataset(xz_train, y_train, z_train)

seeds = [0,1,2,3,4,5,6,7,8,9]
for seed in seeds:
    
    print("< Seed: {} >".format(seed))
    
    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------
    
    model = LogisticRegression(3,1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    # ---------------------
    #  Define FairBatch and DataLoader
    # ---------------------
    
    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = 0.005, target_fairness = 'eqodds', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)
    
    # ---------------------
    #  Model training
    # ---------------------
    for epoch in range(400):

        tmp_loss = []
        
        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)
            
        losses.append(sum(tmp_loss)/len(tmp_loss))
        
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, ED disparity: {}".format(tmp_test['Acc'], tmp_test['EqOdds_diff']))
    print("----------------------------------------------------------------------")

< Seed: 0 >
  Test accuracy: 0.8579999804496765, ED disparity: 0.035440184972895264
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.8550000190734863, ED disparity: 0.04270697728641655
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8560000061988831, ED disparity: 0.035440184972895264
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.8560000061988831, ED disparity: 0.04270697728641655
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8579999804496765, ED disparity: 0.035440184972895264
----------------------------------------------------------------------
< Seed: 5 >
  Test accuracy: 0.8560000061988831, ED disparity: 0.035440184972895264
----------------------------------------------------------------------
< Seed: 6 >
  Test accuracy: 0.8569999933242798, ED disparity: 0.0354401

In [8]:
tmp_acc = []
tmp_ed = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_ed.append(full_tests[i]['EqOdds_diff'])

print("Test accuracy (avg): {}".format(sum(tmp_acc)/len(tmp_acc)))
print("ED disparity  (avg): {}".format(sum(tmp_ed)/len(tmp_ed)))

Test accuracy (avg): 0.8562999963760376
ED disparity  (avg): 0.03762022266695164


# 3. FairBatch w.r.t. Demographic parity
### The results are in Section 4.1 of the paper.

In [7]:
full_tests = []

# Set the train data
train_data = CustomDataset(xz_train, y_train, z_train)

seeds = [0,1,2,3,4,5,6,7,8,9]
for seed in seeds:
    
    print("< Seed: {} >".format(seed))
    
    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------
    
    #model = LogisticRegression(3,1)
    model = nn.Sequential(
        nn.Linear(3, 32),
        nn.ReLU(),
        nn.Linear(32, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
    )

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    # ---------------------
    #  Define FairBatch and DataLoader
    # ---------------------
    
    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = 0.005, target_fairness = 'dp', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    # ---------------------
    #  Model training
    # ---------------------
    for epoch in range(450):

        tmp_loss = []
        
        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)
            
        losses.append(sum(tmp_loss)/len(tmp_loss))
        
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    print("  Test accuracy: {}, DP disparity: {}".format(tmp_test['Acc'], tmp_test['DP_diff']))
    print("----------------------------------------------------------------------")

< Seed: 0 >
  Test accuracy: 0.8130000233650208, DP disparity: 0.10023419203747075
----------------------------------------------------------------------
< Seed: 1 >
  Test accuracy: 0.7889999747276306, DP disparity: 0.06384074941451989
----------------------------------------------------------------------
< Seed: 2 >
  Test accuracy: 0.8410000205039978, DP disparity: 0.12286651053864173
----------------------------------------------------------------------
< Seed: 3 >
  Test accuracy: 0.7929999828338623, DP disparity: 0.06257611241217798
----------------------------------------------------------------------
< Seed: 4 >
  Test accuracy: 0.8399999737739563, DP disparity: 0.1388922716627634
----------------------------------------------------------------------
< Seed: 5 >
  Test accuracy: 0.8040000200271606, DP disparity: 0.06986651053864168
----------------------------------------------------------------------
< Seed: 6 >
  Test accuracy: 0.8730000257492065, DP disparity: 0.202576112412

In [8]:
tmp_acc = []
tmp_dp = []
for i in range(len(seeds)):
    tmp_acc.append(full_tests[i]['Acc'])
    tmp_dp.append(full_tests[i]['DP_diff'])

print("Test accuracy (avg): {}".format(sum(tmp_acc)/len(tmp_acc)))
print("DP disparity  (avg): {}".format(sum(tmp_dp)/len(tmp_dp)))

Test accuracy (avg): 0.8163000047206879
DP disparity  (avg): 0.09841100702576112
